<a href="https://colab.research.google.com/github/GoncaloJoseMoura/Projects/blob/main/youtube_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports/ Downloads

In [ ]:
!pip install youtube_transcript_api
!pip install deepmultilingualpunctuation

In [ ]:
import pandas as pd
import numpy as np

import requests
from youtube_transcript_api import YouTubeTranscriptApi

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import defaultdict

from deepmultilingualpunctuation import PunctuationModel


# Extracting Text

In [ ]:
link = 'https://www.youtube.com/watch?v=zwqKtv5y8Bk'

text = ''
for item in YouTubeTranscriptApi.get_transcript(link.split('=')[-1]):
    text += ' ' + item['text'].lower().strip()

text

" a new day a new python release and my word what a release it is python 3.12 is packed full of improvements and new features so i've scoured around and picked down the most important things you need to know i normally aim for 5 minutes and unders shoot when i've done these before but i imagine i'll probably be making up for that this time as there is a lot to talk about so i suppose let's not waste any time let's start with the big one shall we python speed or supposed like thereof has always been at the center of arguments made by its detractors though many steps have been taken lately to improve the situation python 3.11 bought in optimizations to interpret a startup and of course a specialized adaptive interpreter but 3.12 is going a step further in laying the groundwork for proper multicore support this has been a long time coming quite literally actually in fact support for multiple interpreters the crux of this endeavor has been around since 1997 i was before i was born when it 

# Restauring Punctuation

In [ ]:
from deepmultilingualpunctuation import PunctuationModel

model = PunctuationModel(model="oliverguhr/fullstop-punctuation-multilingual-sonar-base")
text_punctuation = model.restore_punctuation(text)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(


# Summarizing Text

In [ ]:
nlp = spacy.load('en_core_web_sm')

stopwords = list(STOP_WORDS)
doc = nlp(text_punctuation) # applied tokenization
punctuation = punctuation + '\n' # + '‘' + '’' + ' '

word_freq = defaultdict(int)

for word in doc:
    if word.text.lower() not in stopwords and word.text.lower() not in punctuation:
          word_freq[word.text] += 1

In [ ]:
sent_tokens = [sent for sent in doc.sents]

In [ ]:
sent_score = defaultdict(int)

for sent in sent_tokens:
    for word in sent:
        if word.text.lower() in word_freq.keys():
            sent_score[sent] += word_freq[word.text.lower()]

In [ ]:
from heapq import nlargest

perc_to_keep =  0.5

select_length = int(len(sent_score) * perc_to_keep)
select_length

47

In [ ]:
data= []
for key, value in sent_score.items():
    data.append([key.text, value])

# cutting what is most important with ordering, this is now possible since dictionaries are ordered in python 3.7
summarized_text = ' '.join(pd.DataFrame(data).sort_values(1,ascending=False)[0:select_length].sort_index()[0].to_list())

In [ ]:
len(text_punctuation) # original text

8912

In [ ]:
text_punctuation

"a new day, a new python release and my word, what a release it is. python 3.12 is packed full of improvements and new features, so i've scoured around and picked down the most important things you need to know. i normally aim for 5 minutes and unders shoot when i've done these before, but i imagine i'll probably be making up for that this time, as there is a lot to talk about. so i suppose let's not waste any time. let's start with the big one, shall we? python speed, or supposed like thereof, has always been at the center of arguments made by its detractors, though many steps have been taken lately to improve the situation. python 3.11 bought in optimizations to interpret a startup and, of course, a specialized adaptive interpreter. but 3.12 is going a step further in laying the groundwork for proper multicore support. this has been a long time coming, quite literally, actually, in fact, support for multiple interpreters. the crux of this endeavor has been around since 1997. i was be

In [ ]:
len(summarized_text) # summarized text

6558

In [ ]:
summarized_text

"a new day, a new python release and my word, what a release it is. python 3.12 is packed full of improvements and new features, so i've scoured around and picked down the most important things you need to know. i normally aim for 5 minutes and unders shoot when i've done these before, but i imagine i'll probably be making up for that this time, as there is a lot to talk about. python speed, or supposed like thereof, has always been at the center of arguments made by its detractors, though many steps have been taken lately to improve the situation. python 3.11 bought in optimizations to interpret a startup and, of course, a specialized adaptive interpreter. this has been a long time coming, quite literally, actually, in fact, support for multiple interpreters. all interpreters in the same process needed to share a single global state protected by the global interpreter lock or the gill, for short, the gil greatly improved python single core performance at a time when multi-core computa